In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://ppa.launchpad.net/graph

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [14]:
import pandas as pd
d = {'x': [7, 17, 27, 37, 47, 57], 'y': [7, 18, 16, 24, 22, 30], 'z': [17, 12, 19, 24, 22, 45]}
df = pd.DataFrame(data=d)
df1=spark.createDataFrame(df)
df1.show()

+---+---+---+
|  x|  y|  z|
+---+---+---+
|  7|  7| 17|
| 17| 18| 12|
| 27| 16| 19|
| 37| 24| 24|
| 47| 22| 22|
| 57| 30| 45|
+---+---+---+



In [15]:
from pyspark.ml.feature import VectorAssembler
assembleur = VectorAssembler(inputCols=df1.columns, outputCol="caractéristiques")
df_nouv=assembleur.transform(df1)
df_nouv.show()

+---+---+---+----------------+
|  x|  y|  z|caractéristiques|
+---+---+---+----------------+
|  7|  7| 17|  [7.0,7.0,17.0]|
| 17| 18| 12|[17.0,18.0,12.0]|
| 27| 16| 19|[27.0,16.0,19.0]|
| 37| 24| 24|[37.0,24.0,24.0]|
| 47| 22| 22|[47.0,22.0,22.0]|
| 57| 30| 45|[57.0,30.0,45.0]|
+---+---+---+----------------+



In [16]:
from pyspark.ml.feature import MinMaxScaler

scaler = MinMaxScaler (inputCol="caractéristiques", outputCol="mise_à_l_échelle")

scaler_model = scaler.fit(df_nouv)
scaled_data = scaler_model.transform(df_nouv)
scaled_data.select("caractéristiques",'mise_à_l_échelle').show(truncate=False)

+----------------+--------------------------------------------+
|caractéristiques|mise_à_l_échelle                            |
+----------------+--------------------------------------------+
|[7.0,7.0,17.0]  |[0.0,0.0,0.15151515151515152]               |
|[17.0,18.0,12.0]|[0.2,0.4782608695652174,0.0]                |
|[27.0,16.0,19.0]|[0.4,0.391304347826087,0.21212121212121213] |
|[37.0,24.0,24.0]|[0.6,0.7391304347826086,0.36363636363636365]|
|[47.0,22.0,22.0]|[0.8,0.6521739130434783,0.30303030303030304]|
|[57.0,30.0,45.0]|[1.0,1.0,1.0]                               |
+----------------+--------------------------------------------+

